# RTC Product Validation: Point Target Absolute Geolocation Evaluation

Author: Alex Bradley

This notebook uses known corner reflector locations to determine the geometric accuracy of RTC products.

This notebook is a generalisation built on the work of Alex Lewandowski & Franz J Meyer; Alaska Satellite Facility, University of Alaska Fairbanks. 

This notebook has been adapted from :  https://github.com/OPERA-Cal-Val/calval-RTC/blob/main/absolute_geolocation_evaluation/absolute_location_evaluation.ipynb



<hr>

# 1. Load Necessary Libraries

In [ ]:
import csv
import datetime as dt
import math
from pathlib import Path
import re
import requests
import warnings
import os
import re
import ntpath

import asf_search
import fiona 
import geopandas as gpd
import pandas as pd
import lmfit
from lmfit.lineshapes import gaussian2d, lorentzian
from lmfit import Model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyproj
from pyproj import Proj, CRS
import rasterio
from shapely import geometry
from shapely.geometry import Polygon
import shapely.wkt
from urllib.request import urlretrieve
import scipy

# 1. Functions

In [ ]:
# ripping fucntions from isce3 source code to remove reliance on having full project installed...

def estimate_frequency(z):
    # https://github.com/isce-framework/isce3/blob/develop/python/packages/isce3/cal/point_target_info.py
    cx = np.sum(z[:, 1:] * z[:, :-1].conj())
    cy = np.sum(z[1:, :] * z[:-1, :].conj())
    return np.angle([cx, cy])


def shift_frequency(z, fx, fy):
    # https://github.com/isce-framework/isce3/blob/develop/python/packages/isce3/cal/point_target_info.py
    x = np.arange(z.shape[1])
    y = np.arange(z.shape[0])
    z *= np.exp(1j * fx * x)[None,:]
    z *= np.exp(1j * fy * y)[:,None]
    return z

def oversample(x, nov, baseband=False, return_slopes=False):
    # https://github.com/isce-framework/isce3/blob/develop/python/packages/isce3/cal/point_target_info.py
    m, n = x.shape
    assert m == n

    if not baseband:
        # shift the data to baseband
        fx, fy = estimate_frequency(x)
        x = shift_frequency(x, -fx, -fy)

    X = np.fft.fft2(x)
    # Zero-pad high frequencies in the spectrum.
    Y = np.zeros((n * nov, n * nov), dtype=X.dtype)
    n2 = n // 2
    Y[:n2, :n2] = X[:n2, :n2]
    Y[-n2:, -n2:] = X[-n2:, -n2:]
    Y[:n2, -n2:] = X[:n2, -n2:]
    Y[-n2:, :n2] = X[-n2:, :n2]
    # Split Nyquist bins symmetrically.
    assert n % 2 == 0
    Y[:n2, n2] = Y[:n2, -n2] = 0.5 * X[:n2, n2]
    Y[-n2:, n2] = Y[-n2:, -n2] = 0.5 * X[-n2:, n2]
    Y[n2, :n2] = Y[-n2, :n2] = 0.5 * X[n2, :n2]
    Y[n2, -n2:] = Y[-n2, -n2:] = 0.5 * X[n2, -n2:]
    Y[n2, n2] = Y[n2, -n2] = Y[-n2, n2] = Y[-n2, -n2] = 0.25 * X[n2, n2]
    # Back to time domain.
    y = np.fft.ifft2(Y)
    # NOTE account for scaling of different-sized DFTs.
    y *= nov ** 2

    if not baseband:
        # put the phase back on
        y = shift_frequency(y, fx / nov, fy / nov)

    y = np.asarray(y, dtype=x.dtype)
    if return_slopes:
        return (y, fx, fy)
    return y

def oversample_slc(slc,sampling=1,y=None,x=None):
    # https://github.com/OPERA-Cal-Val/calval-RTC/blob/main/absolute_geolocation_evaluation/src/ALE_utils.py
    if y is None:
        y = np.arange(slc.shape[0])
    if x is None:
        x = np.arange(slc.shape[1])

    [rows, cols] = np.shape(slc)
    
    slcovs = oversample(slc,sampling, baseband=True)

    y_orign_step = y[1]-y[0]
    y_ovs_step = y_orign_step/sampling
    x_orign_step = x[1]-x[0]
    x_ovs_step = x_orign_step/sampling

    y = np.arange(y[0],y[-1]+y_orign_step,y_ovs_step)
    x = np.arange(x[0],x[-1]+x_orign_step,x_ovs_step)

    return slcovs,y,x

def findCR(data,y,x,x_bound=[-np.inf,np.inf],y_bound=[-np.inf,np.inf],method="sinc"):
    # https://github.com/OPERA-Cal-Val/calval-RTC/blob/main/absolute_geolocation_evaluation/src/ALE_utils.py
    '''
    Find the location of CR with fitting
    '''
    max_ind = np.argmax(data)
    max_data = data[max_ind]
    
    def _sinc2D(x,x0,y0,a,b,c):
        return c*np.sinc(a*(x[0]-x0))*np.sinc(b*(x[1]-y0))
    
    def _para2D(x,x0,y0,a,b,c,d):
        return a*(x[0]-x0)**2+b*(x[1]-y0)**2+c*(x[0]-x0)*(x[1]-y0)+d

    if method == "sinc":
        # using sinc function for fitting 
        xdata = np.vstack((x,y))
        p0 = [x[max_ind],y[max_ind],0.7,0.7,max_data]
        bounds = ([x_bound[0],y_bound[0],0,0,0],[x_bound[1],y_bound[1],1,1,np.inf])
        popt = scipy.optimize.curve_fit(_sinc2D,xdata,data,p0=p0,bounds=bounds)[0]
        xloc = popt[0]; yloc = popt[1]
    elif method == "para":
        #using paraboloid function for fitting
        xdata = np.vstack((x,y))
        p0 = [x[max_ind],y[max_ind],-1,-1,1,1]
        bounds = ([x_bound[0],y_bound[0],-np.inf,-np.inf,-np.inf,0],[x_bound[1],y_bound[1],0,0,np.inf,np.inf])
        popt = scipy.optimize.curve_fit(_para2D,xdata,data,p0=p0,bounds=bounds)[0]
        xloc = popt[0]; yloc = popt[1]

    return yloc,xloc

def transform_polygon(src_crs, dst_crs, geometry, always_xy=True):
    src_crs = pyproj.CRS(f"EPSG:{src_crs}")
    dst_crs = pyproj.CRS(f"EPSG:{dst_crs}") 
    transformer = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=always_xy)
     # Transform the polygon's coordinates
    transformed_exterior = [
        transformer.transform(x, y) for x, y in geometry.exterior.coords
    ]
    # Create a new Shapely polygon with the transformed coordinates
    transformed_polygon = Polygon(transformed_exterior)
    return transformed_polygon

def assign_crs(tif_path, crs):
    with rasterio.open(tif_path, 'r+') as rds:
        rds.crs = CRS.from_epsg(crs)

<hr>

# 2. Select RTC Products over Validation Sites
These should be a local files that covers a roi where corner reflectors are located


In [ ]:
data_folder = 'data/geolocation'
tif_folder = 'data/tifs'

Australian CRS

In [ ]:
# tif_path = f'{tif_folder}/S1A_IW_20220111T192213_DVP_RTC20_G_gpuned_396E_VV.tif'
# s1_name = 'S1A_IW_SLC__1SDV_20220111T192213_20220111T192240_041417_04ECBD_9F3B'
# tif_path = f'{tif_folder}/S1A_IW_20220123T192213_DVP_RTC20_G_gpuned_2456_VV.tif'
# s1_name = 'S1A_IW_SLC__1SDV_20220123T192213_20220123T192240_041592_04F283_9D0E'
# tif_path = f'{tif_folder}/S1A_IW_20220116T083314_SHP_RTC20_G_gpuned_8BA3_HH.tif'
# s1_name = 'S1A_IW_SLC__1SSH_20220116T083314_20220116T083342_041483_04EED2_DB08'
# ProdID = 'hyp3-gamma'

Maitri Station Antarctica

In [ ]:
#s1_name = 'S1B_IW_SLC__1SSH_20190315T195015_20190315T195045_015369_01CC73_DB8B'

#tif_path = f'{tif_folder}/OPERA_L2_RTC-S1B_IW_SLC__1SSH_20190315T195015_20190315T195045_015369_01CC73_DB8B_HH.tif'
#ProdID = 'rtc-opera'

#tif_path = 'data/tifs/S1B__IW___A_20190315T195015_HH_gamma0-rtc.tif'
#ProdID = 'pyrosar'

#tif_path = 'data/tifs/S1B_IW_20190315T195015_SHP_RTC20_G_gpuned_E650_HH.tif'
#ProdID = 'hyp3-gamma'

Bharati

In [ ]:
s1_name = 'S1B_IW_SLC__1SSH_20190223T222639_20190223T222706_015079_01C2E9_1D63'
tif_path = f'{tif_folder}/32743_OPERA_L2_RTC-S1B_IW_SLC__1SSH_20190223T222639_20190223T222706_015079_01C2E9_1D63_HH.tif'
ProdID = 'rtc-opera-32743'

In [ ]:
# save products to folder
savepath = os.path.join(data_folder,s1_name)
os.makedirs(savepath, exist_ok=True)

dates = s1_name.split('T')[0].split('_')[5]
Year = dates[0:4]
Month = dates[4:6]
Day = dates[6:9]
print(Year, Month, Day)

Reading required file parameters

In [ ]:
if ProdID == 'pyrosar':
    # fix projection in pyrosar tif not assigned correctly
    assign_crs(tif_path, 3031)

# get dif info
with rasterio.open(tif_path) as ds:   
    start_x = ds.transform[2]+0.5*ds.transform[0]
    start_y = ds.transform[5]+0.5*ds.transform[4]
    spacing_x = ds.transform[0]
    spacing_y = ds.transform[4]
    width = ds.profile['width']
    height = ds.profile['height']
    epsg_no = ds.crs.to_epsg()
    b = ds.bounds 
print(epsg_no)

**Visualizing RTC Image**

In [ ]:
with rasterio.open(tif_path) as ds:
    rtc = ds.read(1)

fig, ax = plt.subplots(1, 1, figsize=(10, 10))

ax.set_title(tif_path)
ax.imshow(20*np.log10(np.abs(rtc)), cmap='gray',interpolation=None, origin='upper')

<hr>

# 3. Get Corner Reflector Data and Confirm RTC Coverage

### 3.a Get Corner Reflector data from calval portal

In [ ]:
# download corner reflector dataset from CEOS calval portal
url = 'http://calvalportal.ceos.org/documents/10136/26472/CEOS-reference-targets.kml/be2807cf-2b33-45ca-932f-a91ddc0d2cb1'
cr_filepath = os.path.join(data_folder,'CRS.kml')
urlretrieve(url, cr_filepath)

from fiona.drvsupport import supported_drivers
supported_drivers['LIBKML'] = 'rw'

gdf_list = []
for layer in fiona.listlayers(cr_filepath):    
    gdf = gpd.read_file(cr_filepath, driver='LIBKML', layer=layer)
    gdf_list.append(gdf)

df_cr = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))
df_cr['lon'] = df_cr.geometry.apply(lambda p: p.x if hasattr(p,'x') else np.nan)
df_cr['lat'] = df_cr.geometry.apply(lambda p: p.y if hasattr(p,'y') else np.nan)
print(f'entries: {len(df_cr)}')
df_cr
print('removing entries with no lat lon')
df_cr = df_cr[~(df_cr['lon'].isna() | df_cr['lat'].isna())]
print(f'entries: {len(df_cr)}')
df_cr.head(2)

### 3.b specify corner reflector data

In [ ]:
specify_CRS = True
cr_data = [
    {
        'Name':'Bharati Research Station (Antarctica)',
        'shortname': 'Bharati',
        'lat': -69.404787,
        'lon': 76.190152,
        'slen': 0.9,
        'azm': 350, # unknown, set for descending filter below

    },
    {
        'Name':'Maitri Research Station (Antarctica)',
        'shortname': 'Maitri',
        'lat': -70.767004,
        'lon': 11.72366,
        'slen': 0.9,
        'azm': 100, # unknown, set for ascending filter below

    },
]
if specify_CRS:
    df = pd.DataFrame(cr_data)
    df['geometry'] = df.apply(lambda x: geometry.Point(x.lon, x.lat), axis=1)
    df_cr = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry='geometry')
df_cr    

Discovering which Corner Reflectors are within RTC coverage:

In [ ]:
wkt_border = f'POLYGON(({b.left} {b.top},{b.right} {b.top},{b.right} {b.bottom},{b.left} {b.bottom},{b.left} {b.top}))'
poly = shapely.wkt.loads(wkt_border)
poly_4326 = transform_polygon(epsg_no, 4326, poly, always_xy=False)
print(poly)
print(poly_4326)

#calculating the locations of CRs in SAR image
UTMx = []
UTMy = []
xloc = []
yloc = []
xloc_float = []
yloc_float = []
_in = []

df_cr_target_crs = df_cr.to_crs(epsg_no)

# TODO bad pandas code, fix

for idx, row in df_cr.iterrows():
    
    _Proj = Proj(CRS.from_epsg(epsg_no))
    _x = df_cr_target_crs.iloc[idx].geometry.x
    _y = df_cr_target_crs.iloc[idx].geometry.y
    
    if (np.isfinite(_x) and np.isfinite(_y)):
        # coordinates not finite in utm
    
        #location of CRs in SLC image
        _xloc = int(round((_x-start_x)/spacing_x))    
        _yloc = int(round((_y-start_y)/spacing_y))
        
        UTMx.append(_x) 
        UTMy.append(_y)
        xloc.append(_xloc)
        yloc.append(_yloc)
        xloc_float.append((_x-start_x)/spacing_x)
        yloc_float.append((_y-start_y)/spacing_y)
        _in.append(poly.contains(geometry.Point(_x, _y)))

    else:
        UTMx.append(np.nan) 
        UTMy.append(np.nan)
        xloc.append(np.nan) # loc in image
        yloc.append(np.nan)
        xloc_float.append(np.nan)
        yloc_float.append(np.nan)
        _in.append(np.nan)
    
df_cr['UTMx'] = UTMx
df_cr['UTMy'] = UTMy
df_cr['xloc'] = xloc
df_cr['yloc'] = yloc
df_cr['xloc_float'] = xloc_float
df_cr['yloc_float'] = yloc_float
df_cr['inPoly'] = _in

# exclude non valid crs where values in UTM are not valid
df_cr_roi = df_cr[~df_cr['UTMx'].isna()]

#checking whether CRs are in RTC coverage. Including only CRs within RTC image
df_cr_roi = df_cr_roi[df_cr_roi['inPoly']==True]
df_cr_roi.drop('inPoly', axis=1, inplace=True)
df_cr_roi = df_cr_roi.reset_index(drop=True)
# get the cr size from the description
if 'slen' not in list(df_cr_roi):
    df_cr_roi['slen'] = (df_cr_roi.Name.apply(
        lambda x : float(re.findall("[-+]?(?:\d*\.*\d+) m", x)[0].split('m')[0])))
# add a shortname
if 'shortname' not in list(df_cr_roi):
    df_cr_roi['shortname'] = df_cr_roi['Name'].apply(lambda x : x.split('(')[0])
# set cr direction if not exist
if 'azm' not in list(df_cr_roi):
    df_cr_roi['azm'] = 100 # for ascending pass TODO this is wrong
df_cr_roi.head(2)

**Visualizing** CRs on RTC Image. We color code by reflector size.

In [ ]:
#Displaying RTC image
buffer = 50
minX = df_cr_roi['xloc'].min() - buffer
maxX = df_cr_roi['xloc'].max() + buffer
minY = df_cr_roi['yloc'].min() - buffer
maxY = df_cr_roi['yloc'].max() + buffer

scale_ = 1.0
exp_ = 0.15

fig, ax = plt.subplots(figsize=(15, 7))
cax = ax.imshow(scale_*(np.abs(rtc))**exp_, cmap='gray',interpolation='bilinear', vmin=0.3, vmax=1.7, origin='upper')
ax.set_xlim(minX,maxX)
ax.set_ylim(minY,maxY)
ax.axis('off')

for sl in pd.unique(df_cr_roi.slen):
    xx = df_cr_roi.loc[df_cr_roi['slen']==sl]['xloc']
    yy = df_cr_roi.loc[df_cr_roi['slen']==sl]['yloc']
    ID = df_cr_roi.loc[df_cr_roi['slen']==sl]['shortname']
    
    if sl == 2.4384:
        color=[0.7, 0.7, 0.7]
    elif sl == 4.8:
        color=[0.7, 0.7, 0.7]
    elif sl == 2.8:
        color=[0.7, 0.7, 0.7]
    else:
        color=[0.7, 0.7, 0.7]
    
    ax.scatter(xx,yy,color=color,marker="o",facecolor='none',lw=1)
    for _ID,_xx,_yy in zip(ID,xx,yy):

        ax.annotate(_ID, (_xx+buffer, _yy-buffer), fontsize=10,color=[0.7, 0.7, 0.7])

ax.set_aspect(1)
plt.gca().invert_yaxis()
fig_path = os.path.join(savepath, f'{s1_name}_S1_geoRTC_CRs.png')
fig.savefig(fig_path,dpi=300,bbox_inches='tight')

<hr>

# 4. Remove Corner Reflectors Facing away from the look direction of the S1 Acquisition 

In [ ]:
results = asf_search.granule_search(s1_name)
flight_direction = results[0].properties['flightDirection']
flight_direction

In [ ]:
# selecting CRs according to orbit direction
if flight_direction == 'DESCENDING':
    # descending
    df_filter = df_cr_roi[df_cr_roi['azm']>340].reset_index(drop=True)
    #only east-looking CRs (for right-looking descending)
else:
    # ascending
    df_filter = df_cr_roi[df_cr_roi['azm']<200].reset_index(drop=True)    
    #only west-looking CRs (for right-looking ascending)

df_filter = df_filter.loc[df_filter['slen']>0.8].reset_index(drop=True)   #excluding SWOT CRs (0.7 m as a side length)
df_filter.head(2)

<hr>

# 5. Calculate Absolute Geolocation Error in Easting and Northing

In [ ]:
def lorentzian2d(x, y, amplitude=1., centerx=0., centery=0., sigmax=1., sigmay=1.,
                 rotation=0):
    """Return a two dimensional lorentzian.

    The maximum of the peak occurs at ``centerx`` and ``centery``
    with widths ``sigmax`` and ``sigmay`` in the x and y directions
    respectively. The peak can be rotated by choosing the value of ``rotation``
    in radians.
    """
    xp = (x - centerx)*np.cos(rotation) - (y - centery)*np.sin(rotation)
    yp = (x - centerx)*np.sin(rotation) + (y - centery)*np.cos(rotation)
    R = (xp/sigmax)**2 + (yp/sigmay)**2

    return 2*amplitude*lorentzian(R)/(np.pi*sigmax*sigmay)

def gaussfit(x, y, A, x0, y0, sigma_x, sigma_y, theta):
    theta = np.radians(theta)
    sigx2 = sigma_x**2; sigy2 = sigma_y**2
    a = np.cos(theta)**2/(2*sigx2) + np.sin(theta)**2/(2*sigy2)
    b = np.sin(theta)**2/(2*sigx2) + np.cos(theta)**2/(2*sigy2)
    c = np.sin(2*theta)/(4*sigx2) - np.sin(2*theta)/(4*sigy2)
    
    expo = -a*(x-x0)**2 - b*(y-y0)**2 - 2*c*(x-x0)*(y-y0)
    return A*np.exp(expo) 

In [ ]:
#df_filter = df_filter.iloc[0:5]

In [ ]:
xpeak = []
ypeak = []
snr = []

ovsFactor=32

for ID, xoff, yoff in zip(df_filter['shortname'],df_filter['xloc'],df_filter['yloc']):
    # crop a patch of 10*10 with center at the calculated CR position
    pxbuff = 5
    pybuff = 5
    cropcslc = rtc[(yoff-pybuff):(yoff+pybuff),(xoff-pxbuff):(xoff+pxbuff)]
    
    if np.isnan(np.mean(cropcslc))!=True:
       # _snr = get_snr_peak(cropcslc)

        # find the peak amplitude in the 10*10 patch
        yind,xind = np.unravel_index(np.argmax(np.abs(cropcslc), axis=None), cropcslc.shape)
    
        # give a warning if the peak and the calculated postion are too far
        dyind = yind-pybuff; dxind = xind-pxbuff
        dist = math.sqrt(dyind**2+dxind**2)
        if dist > 2.0:
            warn_str = f'the most bright pixel and the xloc is too far for CR {ID}: {dist:.2f} m'
            warnings.warn(warn_str)
    
        plt.rcParams.update({'font.size': 14})
        fig, ax = plt.subplots(1, 3, figsize=(15, 7))
        ax[0].imshow(np.abs(cropcslc), cmap='gray',interpolation=None, origin='upper')
        ax[0].plot(xind,yind,'r+')
        ax[0].set_title(f'Corner Reflector ID: {ID}', size=8)
    
        # crop a patch of 32*32 but with its center at the peak
        xbuff = 32
        ybuff = 32
        ycrop = np.arange(yoff+dyind-ybuff,yoff+dyind+ybuff)
        xcrop = np.arange(xoff+dxind-xbuff,xoff+dxind+xbuff)
        cropcslc = rtc[ycrop,:][:,xcrop]

        # Oversample slc
        cropcslc_ovs,ycrop_ovs,xcrop_ovs = oversample_slc(cropcslc,sampling=ovsFactor,y=ycrop,x=xcrop)

        numpix = 2
    
        # find the peak amplitude again in a 2 x 2 patch, it correspond to 
        # (2*ovsFactor) x (2*ovsFactor) in oversampled slc
        yoff2 = int(cropcslc_ovs.shape[0]/2)
        xoff2 = int(cropcslc_ovs.shape[1]/2)
        cropcslc2 = cropcslc_ovs[yoff2-numpix*ovsFactor:yoff2+numpix*ovsFactor,
                               xoff2-numpix*ovsFactor:xoff2+numpix*ovsFactor]
        yind2,xind2 = np.unravel_index(np.argmax(abs(cropcslc2), axis=None), cropcslc2.shape)
        dyind2 = yind2-numpix*ovsFactor; dxind2 = xind2-numpix*ovsFactor
    
        N = numpix*2* ovsFactor
        x = np.linspace(0,numpix*2*ovsFactor-1,N)
        y = np.linspace(0,numpix*2*ovsFactor-1,N)
        Xg, Yg = np.meshgrid(x, y)
        fmodel = Model(gaussfit, independent_vars=('x','y'))
        theta = 0.1  # deg
        x0 = numpix* ovsFactor
        y0 = numpix* ovsFactor
        sigx = 2
        sigy = 5
        A = np.max(np.abs(cropcslc2))

        result = fmodel.fit(np.abs(cropcslc2), x=Xg, y=Yg, A=A, x0=x0, y0=y0, sigma_x=sigx, sigma_y=sigy, theta=theta)
        fit = fmodel.func(Xg, Yg, **result.best_values)
    
        dyind3 = result.best_values['y0']-numpix*ovsFactor; dxind3 = result.best_values['x0']-numpix*ovsFactor
    
        ax[1].imshow(np.abs(cropcslc2), cmap='gray',interpolation=None, origin='upper')
        ax[1].plot(xind2,yind2,'r+')
        ax[1].plot(result.best_values['x0'],result.best_values['y0'],'b+')
        ax[1].set_title(f'Oversampled Corner Reflector ID: {ID}', size=8)
    
        ax[2].imshow(np.abs(fit), cmap='gray',interpolation=None, origin='upper')
        ax[2].plot(xind2,yind2,'r+')
        ax[2].plot(result.best_values['x0'],result.best_values['y0'],'b+')
        ax[2].set_title(f'Oversampled Corner Reflector ID: {ID}', size=8)
        fig_path = os.path.join(savepath, f'{s1_name}_{ID}_CR.png')
        plt.savefig(fig_path)
    
        # crop a patch of 3x3 oversampled patch with center at the peak
        cropcslc3 = cropcslc_ovs[yoff2+dyind2-1:yoff2+dyind2+2,xoff2+dxind2-1:xoff2+dxind2+2]
        ycrop2 = ycrop_ovs[yoff2+dyind2-1:yoff2+dyind2+2]
        xcrop2 = xcrop_ovs[xoff2+dxind2-1:xoff2+dxind2+2]
        xxcrop2,yycrop2 = np.meshgrid(xcrop2,ycrop2)
        xxcrop2_f = xxcrop2.flatten()
        yycrop2_f = yycrop2.flatten()
        cropcslc2_f = cropcslc3.flatten()

        # Check if pixel values in a patch are non-NaN
        valid = ~(np.isnan(cropcslc2_f))
        count_valid = np.count_nonzero(valid)

        if count_valid == 0:
            _ypeak, _xpeak = [np.nan, np.nan]

        else:
            _ypeak,_xpeak = findCR(np.abs(cropcslc2_f[valid]),yycrop2_f[valid],xxcrop2_f[valid],
                                x_bound=[xcrop2[0],xcrop2[-1]],y_bound=[ycrop2[0],ycrop2[-1]],method="para")

        #xpeak.append(_xpeak)
        #ypeak.append(_ypeak)
        #xpeak.append(xoff+dxind+dxind2/ovsFactor)
        #ypeak.append(yoff+dyind+dyind2/ovsFactor)
        xpeak.append(xoff+dxind+dxind3/ovsFactor)
        ypeak.append(yoff+dyind+dyind3/ovsFactor)
        #snr.append(_snr)
    else:
        xpeak.append(np.nan)
        ypeak.append(np.nan)
    
df_filter['xloc_CR'] = xpeak
df_filter['yloc_CR'] = ypeak
#df_filter['snr'] = snr
df_filter.head(2)

**Visualizing CR Location Measurements**

In [ ]:
#df_filter = df_filter.dropna().reset_index(drop=True)
df_filter.head(2)

## 5.1 Uncomment this Line to Drop CRs that were Poorly Identified

## 5.2 Calculating Absolute Geolocation Numbers:

In [ ]:
#absloute geolocation error in range and azimuth
ALE_Rg = (df_filter['xloc_CR'] -  df_filter['xloc_float']) * spacing_x
ALE_Az = (df_filter['yloc_CR'] - df_filter['yloc_float']) * spacing_y

test_Rg = ((df_filter['xloc_float'] % 1.0)-0.5)
test_Az = -((df_filter['yloc_float'] % 1.0)-0.5)

test_Rg = ((df_filter['xloc']-df_filter['xloc_float']))
test_Az = -((df_filter['yloc']-df_filter['yloc_float']))

In [ ]:
ALE_Az

In [ ]:
ALE_Rg

In [ ]:
test_Az

In [ ]:
np.sqrt(test_Az**2 + test_Rg**2)

<hr>

## 5.3 Removing Corner Reflectors that are near the Edge of the Pixel


As we cannot interpolate into the pixel, corner reflectors that sit near the edge of the pixel can bias our offset estimate. Therefore, we are removing corner reflectors near pixel edges before we analyze summary statistics. 

In [ ]:
subpix = np.sqrt(test_Az**2 + test_Rg**2)
keepind = []
for idx, row in subpix.items():
    if subpix[idx] <=0.55:
        keepind.append(idx)
print(keepind)

# 6. Plot Absolute Geolocation Error in Easting and Northing

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
requirement = plt.Rectangle((-3.0,-3.0), 6.0, 6.0, fill=False, edgecolor='grey', label='Requirement')
ax.add_patch(requirement)
#sc = ax.scatter(ALE_Rg, ALE_Az, s=200, c=df_filter['slen'], alpha=0.6, marker='o')
sc = ax.scatter(ALE_Rg[keepind], ALE_Az[keepind], s=100, c='k', alpha=0.6, marker='o')
#ax.legend(*sc.legend_elements(),facecolor='lightgray')
#ax.get_legend().set_title('side length (m)')

for ii, txt in enumerate(df_filter.iloc[keepind,0]):
    shortname = df_filter.iloc[keepind[ii]].shortname
    ax.annotate(shortname, (ALE_Rg[keepind[ii]],
                            ALE_Az[keepind[ii]]), 
                            color='black',xytext=(0, 5), 
                            textcoords='offset points',
                            fontsize='8')   #putting IDs in each CR
    
ax.grid(True)
ax.set_xlim(-30,30)
ax.set_ylim(-30,30)
ax.axhline(0, color='black')
ax.axvline(0, color='black')

#np.std(data, ddof=1) / np.sqrt(np.size(data))

ax.set_title(f'Easting: {np.round(np.nanmean(ALE_Rg[keepind]), 3)} +/- {np.round(np.nanstd(ALE_Rg[keepind]) / np.sqrt(np.size(ALE_Rg[keepind])),3)} m, \
    Northing: {np.round(np.nanmean(ALE_Az[keepind]),3)}, +/- {np.round(np.nanstd(ALE_Az[keepind]) / np.sqrt(np.size(ALE_Az[keepind])),3)} m')
ax.set_xlabel('Easting error (m)')
ax.set_ylabel('Northing error (m)')
fig.suptitle('Absolute Geolocation Error')

# plt.errorbar(np.round(np.nanmean(ALE_Rg[keepind]), 3), 
#              np.round(np.nanmean(ALE_Az[keepind]),3),
#              xerr=np.round(np.nanstd(ALE_Rg[keepind]) / np.sqrt(np.size(ALE_Rg[keepind])),3), 
#              yerr=np.round(np.nanstd(ALE_Az[keepind]) / np.sqrt(np.size(ALE_Az[keepind])),3),
#              barsabove=True, capsize=8, capthick=2, fmt='ro', linewidth=2, markersize=10)

output = f"{s1_name}_GeolocationPLOT.png"
fig_path = os.path.join(savepath, output)
plt.savefig(fig_path, dpi=300, transparent='true')

In [ ]:
print(s1_name)
print(np.round(np.nanmean(ALE_Rg[keepind]), 3))
print(np.round(np.nanstd(ALE_Rg[keepind]) / np.sqrt(np.size(ALE_Rg[keepind])),3))
print(np.round(np.nanmean(ALE_Az[keepind]),3))
print(np.round(np.nanstd(ALE_Az[keepind]) / np.sqrt(np.size(ALE_Az[keepind])),3))

In [ ]:
#plotting ALE

#msize = (df_filter['CRZscrores'] - np.min(df_filter['CRZscrores']) + 0.000001) * 100.0

fig, ax = plt.subplots(figsize=(8,8))
requirement = plt.Rectangle((-3.0,-3.0), 6.0, 6.0, fill=False, edgecolor='grey', label='Requirement')
ax.add_patch(requirement)
#sc = ax.scatter(ALE_Rg, ALE_Az, s=200, c=df_filter['slen'], alpha=0.6, marker='o')
sc = ax.scatter(test_Rg, test_Az, s=100, c='k', alpha=0.6, marker='o')


for ii, txt in enumerate(df_filter.iloc[:,0]):
    shortname = df_filter.iloc[ii].shortname
    ax.annotate(shortname, 
                (test_Rg[ii],
                 test_Az[ii]), 
                 color='black',xytext=(0, 5), 
                 textcoords='offset points',
                 fontsize=8)   #putting IDs in each CR
    
ax.grid(True)
ax.set_xlim(-1.25,1.25)
ax.set_ylim(-1.25,1.25)
ax.axhline(0, color='black')
ax.axvline(0, color='black')

ax.set_title(f'Easting: {np.round(np.nanmean(test_Rg), 3)} +/- {np.round(np.nanstd(test_Rg) / np.sqrt(np.size(test_Rg)),3)} m, \
    Northing: {np.round(np.nanmean(test_Az),3)}, +/- {np.round(np.nanstd(test_Az) / np.sqrt(np.size(test_Az)),3)} m')
ax.set_xlabel('Easting error (m)')
ax.set_ylabel('Northing error (m)')
fig.suptitle('Fractional Offset from Pixel Center')

plt.errorbar(np.round(np.nanmean(test_Rg), 3), np.round(np.nanmean(test_Az),3),\
             xerr=np.round(np.nanstd(test_Rg) / np.sqrt(np.size(test_Rg)),3), yerr=np.round(np.nanstd(test_Az) / np.sqrt(np.size(test_Az)),3), \
             barsabove=True, capsize=8, capthick=2, fmt='ro', linewidth=2, markersize=10)

output = f"{s1_name}_FracOffset.png"
fig_path = os.path.join(savepath, output)
plt.savefig(fig_path, dpi=300, transparent='true')

# 7. Write Results into a CSV File

In [ ]:
ALE_csv = os.path.join(savepath,f"{s1_name}_ALE30-Results.csv")

fields = [
    "ProdID"
    "Granule", 
    "Year", 
    "Month", 
    "Day", 
    "Easting_Bias", 
    "sig_Easting_Bias", 
    "Northing_Bias", 
    "sig_Northing_Bias"]

row = [
    ProdID,
    s1_name, 
    Year, 
    Month, 
    Day,  
    np.round(np.nanmean(ALE_Rg[keepind]), 3), 
    np.round(np.nanstd(ALE_Rg[keepind]) / np.sqrt(np.size(ALE_Rg[keepind])),3),
    np.round(np.nanmean(ALE_Az[keepind]), 3), 
    np.round(np.nanstd(ALE_Az[keepind]) / np.sqrt(np.size(ALE_Az[keepind])),3)
    ]

if not os.path.exists(ALE_csv):
    with open(ALE_csv, 'w') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(fields)
        csvwriter.writerow(row)
else:
    with open(ALE_csv, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        s1_names = [c[0] for c in list(csvreader)]
    if s1_name not in s1_names:
        with open(ALE_csv, 'a') as csvfile:
            csvwriter = csv.writer(csvfile)
            csvwriter.writerow(row)

In [ ]:
ALE_csv

*ALE_OPERA_RTC.ipynb - Version 2.0.0 - April 2023*

*Change log*

- Made CR discovery more robust
- Added average visualization in geolocation plot
- Made formatting changes